In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import load_model

# model file path
model = load_model('')
model.summary()

In [ ]:
lines = ### src, tar dataset ###
#lines

In [ ]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:]
#lines.sample(10)

In [ ]:
# <sos> '\t', <eos> '\n'
lines.tar = lines.tar.apply(lambda x : '\t' + x + '\n')
#lines.sample(10)

In [ ]:
src_coordinate = set()
for line in lines.src :
    for i in range(len(line)) :
        src_coordinate.add(line[i])

src_coordinate_size = len(src_coordinate) + 1

encoder_inputs = Input(shape=(None, src_coordinate_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

In [ ]:
# define encoder
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs,
                      outputs=[decoder_inputs] + decoder_states)

index_to_src = dict((i, coordinate) for coordinate, i in src_to_index.items())
index_to_tar = dict((i, coordinate) for coordinate, i in tar_to_index.items())

In [ ]:
def decode_sequence(input_seq):
    # get encoder state from input (map list)
    states_value = encoder_model.predict(input_seq)

    # create <SOS> one-hot vector
    target_seq = np.zeros((1, 1, tar_coordinate_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_route = ""

    # while stop_condition is True
    while not stop_condition:
        # initial state is states_value
        output_coordinate, h, c = decoder_model.predict([target_seq] + states_value)

        # convert prediction to coordinate
        sampled_coordinate_index = np.argmax(output_coordinate[0, -1, :])
        sampled_coordinate = index_to_tar[sampled_coordinate_index]

        # add current predicted coordinate to prediction route
        decoded_route += sampled_coordinate

        # stop when sampled_coordinate reach to <eos> or go over length
        if (sampled_coordinate == '\n' or len(decoded_route) > max_tar_len):
            stop_condition = True

        # store current predicted coordinate for next input
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_coordinate_index] = 1.

        # store current state for next state
        states_value = [h, c]

    return decoded_route

In [ ]:
def predict(input_seq) :
    reverse_target_coordinate_index = dict((i, coordinate) for coordinate, i in tar_to_index.items())
    input_vec = np.zeros(shape=(len(input_seq), max_encoder_seq_length, num_encoder_coordinate), dytpe='float64')
    for i, 

In [ ]:
# enter addresses of start position and destination


In [ ]:
# convert addresses to coordinates


In [ ]:
# get bbox of coordinates to create map array
import numpy as np
import networkx as nx
import osmnx as ox

nodes = []
def bbox(latmax, latmin, lonmax, lonmin) :
    # define a bounding box
    north, south, east, west = latmax, latmin, lonmax, lonmin

    # create network from that bounding box
    # network_type string {"all_private", "all", "bike", "drive", "drive_service", "walk"}
    G = ox.graph_from_bbox(north, south, east, west, network_type="all")
    
    # you can convert your graph to node and edge GeoPandas GeoDataFrames
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
    
    # (latitude, longtitude) list
    for i in range(len(gdf_nodes)) :
        nodes.append([gdf_nodes['y'].values[i], gdf_nodes['x'].values[i]])
    
    return nodes

In [ ]:
# model predict


In [ ]:
# show predicted route
